## Carregando as Bases de dados

In [1]:
# imports

import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


In [2]:
# FUNÇÃO PARA VERIFICAR A QUANTIDADE DE DADOS FALTANTES

def missing_data_info(data):
    missing_count = data.isnull().sum()
    missing_percent = 100 * data.isnull().mean()
    data_types = data.dtypes
    missing_info = pd.DataFrame({
        'Missing Count': missing_count,
        'Missing Percent': missing_percent,
        'Data Type': data_types
    })

    return missing_info.sort_values(by='Missing Percent', ascending=False)

# missing_data_info(df)

## Preparação da base

In [3]:
# Carregando o dataset FAMILIAS/pessoas

pd.set_option('display.max_columns', 51)
pd.set_option('display.max_rows', 51)

df = pd.read_csv('bases/df_fam_pessoas_amostra_100mil.csv', sep=',')
print(f'Shape:', df.shape)

df.head()

Shape: (267140, 51)


,cd_ibge,uf_ibge,regiao_ibge,estrato,classf,id_familia,vlr_renda_media_fam,cod_local_domic_fam,cod_especie_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,cod_agua_canalizada_fam,cod_abaste_agua_domic_fam,cod_banheiro_domic_fam,cod_escoa_sanitario_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,cod_familia_indigena_fam,ind_familia_quilombola_fam,ind_parc_mds_fam,qtde_pessoas,peso.fam,dias_cadastramento,dias_atualizacao,classe_renda,id_pessoa,cod_sexo_pessoa,idade,cod_parentesco_rf_pessoa,cod_raca_cor_pessoa,cod_local_nascimento_pessoa,cod_certidao_registrada_pessoa,cod_deficiencia_memb,cod_sabe_ler_escrever_memb,ind_frequenta_escola_memb,cod_escola_local_memb,cod_curso_frequenta_memb,cod_ano_serie_frequenta_memb,cod_curso_frequentou_pessoa_memb,cod_ano_serie_frequentou_memb,cod_concluiu_frequentou_memb,cod_trabalhou_memb,cod_afastado_trab_memb,cod_agricultura_trab_memb,cod_principal_trab_memb,cod_trabalho_12_meses_memb,qtd_meses_12_meses_memb,peso.pes
0,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,2351897,2,4,3,1.0,2.0,1,2,2.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563277716385694
1,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,2578989,2,15,3,1.0,1.0,1,2,1.0,1.0,1.0,7.0,1.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,563277716385694
2,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,3312847,2,13,3,1.0,1.0,1,2,2.0,1.0,1.0,5.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563277716385694
3,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,10482075,2,38,1,1.0,2.0,1,2,1.0,3.0,NaN,NaN,NaN,4.0,4.0,1.0,1.0,NaN,1.0,6.0,1.0,12.0,563277716385694
4,2931905,29,2,2,3,2888440,0.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,4.0,1.0,3.0,3.0,1.0,3.0,2.0,2.0,0.0,2,550282046697327,2646.0,151,0.0,5553031,2,5,3,4.0,2.0,1,2,2.0,1.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563263195326236


In [4]:
df.columns

Index(['cd_ibge', 'uf_ibge', 'regiao_ibge', 'estrato', 'classf', 'id_familia',
       'vlr_renda_media_fam', 'cod_local_domic_fam', 'cod_especie_domic_fam',
       'qtd_comodos_domic_fam', 'qtd_comodos_dormitorio_fam',
       'cod_material_piso_fam', 'cod_material_domic_fam',
       'cod_agua_canalizada_fam', 'cod_abaste_agua_domic_fam',
       'cod_banheiro_domic_fam', 'cod_escoa_sanitario_domic_fam',
       'cod_destino_lixo_domic_fam', 'cod_iluminacao_domic_fam',
       'cod_calcamento_domic_fam', 'cod_familia_indigena_fam',
       'ind_familia_quilombola_fam', 'ind_parc_mds_fam', 'qtde_pessoas',
       'peso.fam', 'dias_cadastramento', 'dias_atualizacao', 'classe_renda',
       'id_pessoa', 'cod_sexo_pessoa', 'idade', 'cod_parentesco_rf_pessoa',
       'cod_raca_cor_pessoa', 'cod_local_nascimento_pessoa',
       'cod_certidao_registrada_pessoa', 'cod_deficiencia_memb',
       'cod_sabe_ler_escrever_memb', 'ind_frequenta_escola_memb',
       'cod_escola_local_memb', 'cod_curso_frequ

In [5]:
# DELETANDO COLUNAS DESNECESSÁRIAS
print(f'Shape antes:', df.shape)
df.drop('cd_ibge', axis=1, inplace=True)
df.drop('regiao_ibge', axis=1, inplace=True)
df.drop('vlr_renda_media_fam', axis=1, inplace=True)
df.drop('peso.fam', axis=1, inplace=True)
df.drop('peso.pes', axis=1, inplace=True)
print(f'Shape depois:', df.shape)

Shape antes: (267140, 51)
Shape depois: (267140, 46)


In [6]:
# CRIANDO LISTA DE VARIÁVEIS POR TEMA PARA FACILITAR MANIPULAÇÃO E TRATAMENTO

var_id = ['id_familia', 'id_pessoa']
var_target = ['classe_renda']
var_educ = ['qtde_pessoas', 'idade', 'cod_sabe_ler_escrever_memb', 'ind_frequenta_escola_memb',
            'cod_ano_serie_frequenta_memb', 'cod_escola_local_memb', 'cod_curso_frequenta_memb',
       'cod_ano_serie_frequenta_memb', 'cod_curso_frequentou_pessoa_memb',
       'cod_ano_serie_frequentou_memb', 'cod_concluiu_frequentou_memb']


## Tratando variáveis educação

In [7]:
# filtrando base
selected_columns = var_id + var_educ
df1 = df[selected_columns]

df1

,id_familia,id_pessoa,qtde_pessoas,idade,cod_sabe_ler_escrever_memb,ind_frequenta_escola_memb,cod_ano_serie_frequenta_memb,cod_escola_local_memb,cod_curso_frequenta_memb,cod_ano_serie_frequenta_memb,cod_curso_frequentou_pessoa_memb,cod_ano_serie_frequentou_memb,cod_concluiu_frequentou_memb
0,268503,2351897,4,4,2.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN
1,268503,2578989,4,15,1.0,1.0,1.0,1.0,7.0,1.0,NaN,NaN,NaN
2,268503,3312847,4,13,2.0,1.0,7.0,1.0,5.0,7.0,NaN,NaN,NaN
3,268503,10482075,4,38,1.0,3.0,NaN,NaN,NaN,NaN,4.0,4.0,1.0
4,2888440,5553031,2,5,2.0,1.0,1.0,1.0,5.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267135,1481101,4367533,2,65,1.0,3.0,NaN,NaN,NaN,NaN,4.0,2.0,2.0
267136,1481101,8608500,2,53,1.0,3.0,NaN,NaN,NaN,NaN,4.0,4.0,1.0
267137,2934752,4274948,4,48,1.0,3.0,NaN,NaN,NaN,NaN,4.0,4.0,2.0
267138,2934752,6291827,4,15,1.0,1.0,9.0,1.0,5.0,9.0,NaN,NaN,NaN


In [8]:
df1.columns

Index(['id_familia', 'id_pessoa', 'qtde_pessoas', 'idade',
       'cod_sabe_ler_escrever_memb', 'ind_frequenta_escola_memb',
       'cod_ano_serie_frequenta_memb', 'cod_escola_local_memb',
       'cod_curso_frequenta_memb', 'cod_ano_serie_frequenta_memb',
       'cod_curso_frequentou_pessoa_memb', 'cod_ano_serie_frequentou_memb',
       'cod_concluiu_frequentou_memb'],
      dtype='object')

In [9]:
df1['cod_curso_frequenta_memb'].value_counts(normalize=True)

cod_curso_frequenta_memb
5.0     0.596566
7.0     0.141817
2.0     0.075724
1.0     0.061267
4.0     0.050347
13.0    0.018744
3.0     0.018534
10.0    0.014468
11.0    0.009179
6.0     0.005025
8.0     0.004386
9.0     0.003537
12.0    0.000309
14.0    0.000099
Name: proportion, dtype: float64

In [10]:
# CRIANDO VARIAVEIS CARACTERÍSTICAS DE EDUCACAO NA FAMILIA

df_var_educacao = df.groupby('id_familia').apply(
    lambda x: pd.Series({
        'pct_analfabetos>10': ((x['cod_sabe_ler_escrever_memb'] == 2) & (x['idade'] > 10)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_jovens_sem_escola': ((x['idade'] >= 6) & (x['idade'] < 18) & x['ind_frequenta_escola_memb'].isin([3, 4])).sum() / x['qtde_pessoas'].iloc[0],
        'pct_jovens_escola_privada': ((x['idade'] >= 6) & (x['idade'] < 18) & x['ind_frequenta_escola_memb'] == 2).sum() / x['qtde_pessoas'].iloc[0],
        'pct_criancas_creche': ((x['idade'] < 6) & x['cod_curso_frequenta_memb'].isin([1, 2])).sum() / x['qtde_pessoas'].iloc[0],
        'pct_fundamental1': (x['cod_curso_frequentou_pessoa_memb'] == 4).sum() / x['qtde_pessoas'].iloc[0],
        'pct_fundamental2': (x['cod_curso_frequentou_pessoa_memb'] == 5).sum() / x['qtde_pessoas'].iloc[0],
        'pct_ensino_medio': (x['cod_curso_frequentou_pessoa_memb'] == 8).sum() / x['qtde_pessoas'].iloc[0],
    }),
    include_groups=False
).reset_index()

df_var_educacao


,id_familia,pct_analfabetos>10,pct_jovens_sem_escola,pct_jovens_escola_privada,pct_criancas_creche,pct_fundamental1,pct_fundamental2,pct_ensino_medio
0,14,0.333333,0.00,0.0,0.000000,0.000000,0.000000,0.666667
1,36,0.000000,0.00,0.0,0.000000,0.000000,0.250000,0.250000
2,57,0.000000,0.25,0.0,0.000000,0.000000,0.000000,1.000000
3,65,0.000000,0.00,0.0,0.000000,0.500000,0.250000,0.000000
4,124,0.000000,0.00,0.0,0.333333,0.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...
99995,5290445,0.000000,0.00,0.0,0.000000,0.333333,0.000000,0.000000
99996,5290509,0.000000,0.00,0.0,0.333333,0.000000,0.333333,0.000000
99997,5290542,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.250000
99998,5290562,0.000000,0.00,0.0,0.000000,1.000000,0.000000,0.000000


In [11]:
# ANALISANDO DADOS FALTANTES
missing_data_info(df_var_educacao)

,Missing Count,Missing Percent,Data Type
id_familia,0,0.0,int64
pct_analfabetos>10,0,0.0,float64
pct_jovens_sem_escola,0,0.0,float64
pct_jovens_escola_privada,0,0.0,float64
pct_criancas_creche,0,0.0,float64
pct_fundamental1,0,0.0,float64
pct_fundamental2,0,0.0,float64
pct_ensino_medio,0,0.0,float64


In [12]:
# Salvando base
df_var_educacao.to_csv('base_pre/df_var_educacao.csv', index=False)
